
# HI-VAE 对齐实验：TensorFlow 参考实现 vs SUAVE (behaviour="hivae")

本笔记展示如何在 `mimic-example-1000.tsv` 数据集上分别运行 TensorFlow 版 HI-VAE 与 SUAVE（`behaviour="hivae"`），并比较缺失值填补以及其他主要输出是否保持一致。



> ⚠️ **TensorFlow 版本要求**：`third_party/hivae_tf` 官方 README 指定需要 TensorFlow 2.x。在运行下方代码前，请确保已经安装兼容版本，例如：``pip install 'tensorflow==2.11.*'``。


In [1]:

import os
from pathlib import Path
import math
import warnings

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")
import tensorflow as tf

from suave.model import SUAVE
from suave.types import Schema
from third_party.hivae_tf.hivae import hivae as tf_hivae

assert tf.__version__.startswith('2.'), "TensorFlow 2.x is required for the TF HI-VAE reference implementation."
print(f'TensorFlow version: {tf.__version__}')


2025-09-18 13:51:06.575518: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-18 13:51:06.575651: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-18 13:51:06.586697: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/root/.pyenv/versions/3.11.12/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TensorFlow version: 2.15.1


In [2]:

RANDOM_SEED = 7
DATA_PATH = Path('examples/data/mimic_for_test/mimic-septic_shock.tsv')
assert DATA_PATH.exists(), f'Expected data file not found: {DATA_PATH}'
raw_df = pd.read_csv(DATA_PATH, sep='	')
print(f'Dataset shape: {raw_df.shape}')
raw_df.head()


Dataset shape: (4623, 37)


,Unnamed: 0,age,sex,BMI,temperature,heart_rate,respir_rate,SBP,DBP,MAP,...,PT,APTT,PH,PaO2,PaO2/FiO2,PaCO2,HCO3-,Lac,3d_septic_shock,7d_septic_shock
0,29349,26,0,NaN,37.748750,85.520000,16.370370,110.269231,63.538462,80.096154,...,14.45,25.75,7.480,302.0,503.333333,25.0,17.0,NaN,0,0
1,25936,76,0,26.271218,37.083333,81.041667,17.900000,160.208333,62.500000,87.250000,...,12.60,53.00,7.265,51.5,128.750000,42.0,18.5,1.4,0,0
2,9686,76,1,35.967860,37.498571,86.360000,18.220000,109.500000,62.291667,75.958333,...,11.70,25.10,7.195,107.5,134.166667,38.5,14.5,1.4,0,0
3,11918,71,1,NaN,37.412857,110.851852,22.555556,105.440000,56.720000,69.800000,...,16.15,33.20,NaN,NaN,NaN,NaN,19.5,NaN,0,0
4,26289,78,1,23.979239,36.697143,94.307692,20.961538,98.015625,53.171875,69.843750,...,22.30,37.25,7.435,102.0,NaN,31.0,20.0,1.7,0,0


In [3]:

summary = pd.DataFrame({
    'dtype': raw_df.dtypes.astype(str),
    'n_unique': raw_df.nunique(dropna=True),
    'missing_ratio': raw_df.isna().mean(),
}).sort_index()
summary


,dtype,n_unique,missing_ratio
3d_septic_shock,int64,2,0.000000
7d_septic_shock,int64,2,0.000000
ALT,float64,498,0.327277
APTT,float64,1035,0.074194
AST,float64,577,0.320787
BMI,float64,1917,0.506597
BUN,float64,247,0.005408
CRRT,int64,2,0.000000
DBP,float64,3569,0.000216
Fg,float64,608,0.741942


In [4]:

DEFAULT_CATEGORICAL = [
    'sex',
    'CRRT',
    'Respiratory_Support',
    '3d_septic_shock',
    '7d_septic_shock',
]
DEFAULT_ORDINAL = ['SOFA_cns']
object_columns = [col for col in raw_df.columns if raw_df[col].dtype == 'object']
categorical_columns = sorted({*DEFAULT_CATEGORICAL, *object_columns} & set(raw_df.columns))
ordinal_columns = [col for col in DEFAULT_ORDINAL if col in raw_df.columns]
print('Categorical columns (editable):', categorical_columns)
print('Ordinal columns (editable):', ordinal_columns)


Categorical columns (editable): ['3d_septic_shock', '7d_septic_shock', 'CRRT', 'Respiratory_Support', 'sex']
Ordinal columns (editable): ['SOFA_cns']


In [5]:

from typing import Dict, Iterable, Mapping, Sequence

def _prepare_dataframes(
    df: pd.DataFrame,
    categorical: Sequence[str],
    ordinal: Sequence[str],
) -> tuple[pd.DataFrame, pd.DataFrame, Dict[str, list], Dict[str, list]]:
    df_suave = df.copy()
    df_hivae = df.copy()
    cat_categories: Dict[str, list] = {}
    ord_categories: Dict[str, list] = {}

    for column in categorical:
        if column not in df_suave.columns:
            continue
        categories = pd.Index(sorted(df_suave[column].dropna().unique()))
        if len(categories) <= 1:
            warnings.warn(
                f"Column '{column}' has <=1 observed category; treating as real-valued.",
                UserWarning,
            )
            continue
        cat_categories[column] = categories.tolist()
        suave_series = pd.Categorical(df_suave[column], categories=categories)
        df_suave[column] = suave_series
        codes = pd.Series(suave_series.codes, index=df_suave.index, dtype=float)
        df_hivae[column] = codes.replace(-1, np.nan)

    for column in ordinal:
        if column not in df_suave.columns:
            continue
        categories = pd.Index(sorted(df_suave[column].dropna().unique()))
        if len(categories) <= 1:
            warnings.warn(
                f"Column '{column}' has <=1 observed level; treating as real-valued.",
                UserWarning,
            )
            continue
        ord_categories[column] = categories.tolist()
        suave_series = pd.Categorical(df_suave[column], categories=categories, ordered=True)
        df_suave[column] = suave_series
        codes = pd.Series(suave_series.codes, index=df_suave.index, dtype=float)
        df_hivae[column] = codes.replace(-1, np.nan)

    remaining = [
        col for col in df.columns
        if col not in set(cat_categories) and col not in set(ord_categories)
    ]
    df_suave[remaining] = df[remaining]
    df_hivae[remaining] = df[remaining]
    return df_suave, df_hivae, cat_categories, ord_categories

def _build_schema_and_types(
    columns: Iterable[str],
    categorical: Mapping[str, Sequence],
    ordinal: Mapping[str, Sequence],
) -> tuple[Schema, list]:
    schema_dict: Dict[str, Mapping[str, object]] = {}
    types_list: list = []
    for column in columns:
        if column in categorical:
            n_classes = len(categorical[column])
            schema_dict[column] = {'type': 'cat', 'n_classes': n_classes}
            types_list.append((column, 'cat', n_classes, n_classes))
        elif column in ordinal:
            n_classes = len(ordinal[column])
            schema_dict[column] = {'type': 'ordinal', 'n_classes': n_classes}
            types_list.append((column, 'ordinal', n_classes, n_classes))
        else:
            schema_dict[column] = {'type': 'real'}
            types_list.append((column, 'real', 1, None))
    schema = Schema(schema_dict)
    return schema, types_list


In [6]:

suave_df, hivae_df, cat_maps, ord_maps = _prepare_dataframes(raw_df, categorical_columns, ordinal_columns)
schema, types_list = _build_schema_and_types(raw_df.columns, cat_maps, ord_maps)
print(schema.to_dict())
print('Number of HI-VAE features:', len(types_list))
suave_df.head()


{'Unnamed: 0': {'type': 'real'}, 'age': {'type': 'real'}, 'sex': {'type': 'cat', 'n_classes': 2}, 'BMI': {'type': 'real'}, 'temperature': {'type': 'real'}, 'heart_rate': {'type': 'real'}, 'respir_rate': {'type': 'real'}, 'SBP': {'type': 'real'}, 'DBP': {'type': 'real'}, 'MAP': {'type': 'real'}, 'SOFA_cns': {'type': 'ordinal', 'n_classes': 5}, 'CRRT': {'type': 'cat', 'n_classes': 2}, 'Respiratory_Support': {'type': 'cat', 'n_classes': 5}, 'WBC': {'type': 'real'}, 'Hb': {'type': 'real'}, 'NE%': {'type': 'real'}, 'LYM%': {'type': 'real'}, 'PLT': {'type': 'real'}, 'ALT': {'type': 'real'}, 'AST': {'type': 'real'}, 'STB': {'type': 'real'}, 'BUN': {'type': 'real'}, 'Scr': {'type': 'real'}, 'Glu': {'type': 'real'}, 'K+': {'type': 'real'}, 'Na+': {'type': 'real'}, 'Fg': {'type': 'real'}, 'PT': {'type': 'real'}, 'APTT': {'type': 'real'}, 'PH': {'type': 'real'}, 'PaO2': {'type': 'real'}, 'PaO2/FiO2': {'type': 'real'}, 'PaCO2': {'type': 'real'}, 'HCO3-': {'type': 'real'}, 'Lac': {'type': 'real'}, 

,Unnamed: 0,age,sex,BMI,temperature,heart_rate,respir_rate,SBP,DBP,MAP,...,PT,APTT,PH,PaO2,PaO2/FiO2,PaCO2,HCO3-,Lac,3d_septic_shock,7d_septic_shock
0,29349,26,0,NaN,37.748750,85.520000,16.370370,110.269231,63.538462,80.096154,...,14.45,25.75,7.480,302.0,503.333333,25.0,17.0,NaN,0,0
1,25936,76,0,26.271218,37.083333,81.041667,17.900000,160.208333,62.500000,87.250000,...,12.60,53.00,7.265,51.5,128.750000,42.0,18.5,1.4,0,0
2,9686,76,1,35.967860,37.498571,86.360000,18.220000,109.500000,62.291667,75.958333,...,11.70,25.10,7.195,107.5,134.166667,38.5,14.5,1.4,0,0
3,11918,71,1,NaN,37.412857,110.851852,22.555556,105.440000,56.720000,69.800000,...,16.15,33.20,NaN,NaN,NaN,NaN,19.5,NaN,0,0
4,26289,78,1,23.979239,36.697143,94.307692,20.961538,98.015625,53.171875,69.843750,...,22.30,37.25,7.435,102.0,NaN,31.0,20.0,1.7,0,0


In [7]:

train_idx, test_idx = train_test_split(
    suave_df.index, test_size=0.2, random_state=RANDOM_SEED, shuffle=True
)
train_idx = sorted(train_idx)
test_idx = sorted(test_idx)
train_suave = suave_df.loc[train_idx].reset_index(drop=True)
test_suave = suave_df.loc[test_idx].reset_index(drop=True)
train_hivae = hivae_df.loc[train_idx].reset_index(drop=True)
test_hivae = hivae_df.loc[test_idx].reset_index(drop=True)
test_reference = raw_df.loc[test_idx].reset_index(drop=True)
print(f'Train shape: {train_suave.shape}, Test shape: {test_suave.shape}')


Train shape: (3698, 37), Test shape: (925, 37)


In [8]:

def introduce_artificial_missing(df: pd.DataFrame, rate: float, *, seed: int) -> tuple[pd.DataFrame, np.ndarray]:
    rng = np.random.default_rng(seed)
    observed = ~df.isna().to_numpy()
    mask = np.zeros(df.shape, dtype=bool)
    corrupted = df.copy()
    for j, column in enumerate(df.columns):
        candidates = np.where(observed[:, j])[0]
        if len(candidates) == 0:
            continue
        n_remove = max(1, int(round(rate * len(candidates))))
        n_remove = min(n_remove, len(candidates))
        selected = rng.choice(candidates, size=n_remove, replace=False)
        corrupted.iloc[selected, j] = np.nan
        mask[selected, j] = True
    return corrupted, mask

def evaluate_imputations(
    original: pd.DataFrame,
    imputed: pd.DataFrame,
    mask: np.ndarray,
    discrete_columns: Sequence[str],
) -> pd.DataFrame:
    records = []
    columns = list(original.columns)
    for j, column in enumerate(columns):
        column_mask = mask[:, j]
        if not column_mask.any():
            continue
        truth = original.loc[column_mask, column]
        preds = imputed.loc[column_mask, column]
        if column in discrete_columns:
            truth_values = truth.astype(str).reset_index(drop=True)
            pred_values = preds.astype(str).reset_index(drop=True)
            accuracy = float((truth_values == pred_values).mean())
            records.append({
                'column': column,
                'metric': 'categorical_accuracy',
                'value': accuracy,
            })
        else:
            truth_numeric = pd.to_numeric(truth, errors='coerce')
            preds_numeric = pd.to_numeric(preds, errors='coerce')
            diff = preds_numeric - truth_numeric
            if np.all(np.isnan(diff)):
                continue
            mse = float(np.nanmean(np.square(diff)))
            records.append({
                'column': column,
                'metric': 'mse',
                'value': mse,
            })
    return pd.DataFrame(records)


In [9]:

artificial_rate = 0.1
test_suave_corrupted, artificial_mask = introduce_artificial_missing(
    test_suave, rate=artificial_rate, seed=RANDOM_SEED
)
test_hivae_corrupted = test_hivae.copy()
for j, column in enumerate(test_hivae.columns):
    missing_indices = np.where(artificial_mask[:, j])[0]
    if len(missing_indices):
        test_hivae_corrupted.iloc[missing_indices, j] = np.nan
print('Corrupted test set ready. Artificially masked cells:', artificial_mask.sum())


Corrupted test set ready. Artificially masked cells: 2976


In [10]:

EPOCHS = 25
BATCH_SIZE = min(128, len(train_suave))
suave_model = SUAVE(
    schema=schema,
    behaviour='hivae',
    latent_dim=32,
    hidden_dims=(128, 64),
    batch_size=BATCH_SIZE,
    random_state=RANDOM_SEED,
)
suave_model.fit(train_suave, epochs=EPOCHS, batch_size=BATCH_SIZE)


CUDA not available; falling back to CPU


Training:   0% 0/25 [00:00<?, ?it/s]

Training:   0% 0/25 [00:01<?, ?it/s, loss=112]

Training:   4% 1/25 [00:01<00:29,  1.23s/it, loss=112]

Training:   4% 1/25 [00:02<00:29,  1.23s/it, loss=110]

...

Training:  92% 23/25 [00:33<00:03,  1.61s/it, loss=106]

Training:  96% 24/25 [00:33<00:01,  1.63s/it, loss=106]

Training:  96% 24/25 [00:35<00:01,  1.63s/it, loss=107]

Training: 100% 25/25 [00:35<00:00,  1.64s/it, loss=107]

SUAVE(latent_dim=32, beta=1.5, hidden_dims=(128, 64), dropout=0.1, learning_rate=0.001)

In [11]:

cache_dir = Path('examples/.cache/mimic_hivae_tf')
cache_dir.mkdir(parents=True, exist_ok=True)
network_dir = cache_dir / 'networks'
results_dir = cache_dir / 'results'
network_dir.mkdir(exist_ok=True)
results_dir.mkdir(exist_ok=True)

hivae_config = {
    'batch_size': min(64, len(train_hivae)),
    'model_name': 'model_HIVAE_inputDropout',
    'dim_z': 32,
    'dim_y': 32,
    'dim_s': 32,
}
hivae_model = tf_hivae(
    types_list,
    hivae_config,
    results_path=results_dir,
    network_path=network_dir,
    verbosity_level=0,
)
train_true_mask = (~train_hivae.isna()).astype(int)
hivae_model.fit(train_hivae, epochs=EPOCHS, true_missing_mask=train_true_mask)


DEBUG:   	 1 2 self.full_network_path examples/.cache/mimic_hivae_tf/networks/model_HIVAE_inputDropout_s32_z32_y32_batch64_0216dbd3-0e2c-419c-ac55-205bb0f3fa23
DEBUG:   	 1 2 self.full_results_path examples/.cache/mimic_hivae_tf/results/model_HIVAE_inputDropout_s32_z32_y32_batch64_0216dbd3-0e2c-419c-ac55-205bb0f3fa23
DEBUG:   	 1 2 self.network_file_name examples/.cache/mimic_hivae_tf/networks/model_HIVAE_inputDropout_s32_z32_y32_batch64_0216dbd3-0e2c-419c-ac55-205bb0f3fa23_ckpt
DEBUG:   	 1 2 hivae:_training:len(training_data) 3698
DEBUG:   	 1 2 [*] Importing model: model_HIVAE_inputDropout
DEBUG:   	 1 2 [*] Defining placeholders


DEBUG:   	 1 2 [*] Defining Encoder...
DEBUG:   	 1 2 [*] Defining Decoder...


/workspace/SUAVE/third_party/hivae_tf/hivae/VAE_functions.py:117: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  log_pi = tf.compat.v1.layers.dense(inputs=X, units=s_dim, activation=None,
/workspace/SUAVE/third_party/hivae_tf/hivae/VAE_functions.py:134: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  mean_qz = tf.compat.v1.layers.dense(inputs=tf.concat([X,samples_s],1), units=z_dim, activation=None,
/workspace/SUAVE/third_party/hivae_tf/hivae/VAE_functions.py:136: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  log_var_qz = tf.compat.v1.layers.dense(inputs=tf.concat([X,samples_s],1), units=z_dim, activation=None,
/workspace/SUAVE/third_party/hivae_tf/hivae/VAE_functions.py:211: UserWarning: `tf.layers.dense` is deprecated and will be rem

/workspace/SUAVE/third_party/hivae_tf/hivae/VAE_functions.py:433: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  obs_output = tf.compat.v1.layers.dense(inputs=observed_data, units=output_dim, activation=None,
/workspace/SUAVE/third_party/hivae_tf/hivae/VAE_functions.py:435: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  miss_output = tf.compat.v1.layers.dense(inputs=missing_data, units=output_dim, activation=None,


DEBUG:   	 1 2 [*] Defining Cost function...


...

DEBUG:   	 1 2 Clusters: 12
DEBUG:   	 1 2 Epoch:   21	time: 65.94	train_loglik: -98.78	KL_z:  2.67	KL_s:  2.70	ELBO: -104.15	Test_loglik: 100000.00


DEBUG:   	 1 2 Clusters: 12
DEBUG:   	 1 2 Epoch:   22	time: 68.53	train_loglik: -98.70	KL_z:  2.69	KL_s:  2.71	ELBO: -104.10	Test_loglik: 100000.00


DEBUG:   	 1 2 Clusters: 12


DEBUG:   	 1 2 Epoch:   23	time: 71.79	train_loglik: -98.71	KL_z:  2.73	KL_s:  2.72	ELBO: -104.16	Test_loglik: 100000.00


DEBUG:   	 1 2 Clusters: 12
DEBUG:   	 1 2 Epoch:   24	time: 75.62	train_loglik: -98.60	KL_z:  2.74	KL_s:  2.73	ELBO: -104.07	Test_loglik: 100000.00
INFO:    	 0 3 Training Finished ...
DEBUG:   	 1 2 Saving informations ...
DEBUG:   	 1 2 Saving model_HIVAE_inputDropout_s32_z32_y32_batch64_0216dbd3-0e2c-419c-ac55-205bb0f3fa23_loglik.csv in examples/.cache/mimic_hivae_tf/results/model_HIVAE_inputDropout_s32_z32_y32_batch64_0216dbd3-0e2c-419c-ac55-205bb0f3fa23
DEBUG:   	 1 2 Saving model_HIVAE_inputDropout_s32_z32_y32_batch64_0216dbd3-0e2c-419c-ac55-205bb0f3fa23_KL_s.csv in examples/.cache/mimic_hivae_tf/results/model_HIVAE_inputDropout_s32_z32_y32_batch64_0216dbd3-0e2c-419c-ac55-205bb0f3fa23
DEBUG:   	 1 2 Saving model_HIVAE_inputDropout_s32_z32_y32_batch64_0216dbd3-0e2c-419c-ac55-205bb0f3fa23_KL_z.csv in examples/.cache/mimic_hivae_tf/results/model_HIVAE_inputDropout_s32_z32_y32_batch64_0216dbd3-0e2c-419c-ac55-205bb0f3fa23
DEBUG:   	 1 2 Saving model_HIVAE_inputDropout_s32_z32_y32_bat

In [12]:

suave_imputed = suave_model.impute(test_suave_corrupted, only_missing=False)
suave_decoded = suave_imputed.copy()
for column, categories in cat_maps.items():
    if column in suave_decoded.columns:
        suave_decoded[column] = suave_decoded[column].astype(str)
for column, categories in ord_maps.items():
    if column in suave_decoded.columns:
        suave_decoded[column] = suave_decoded[column].astype(str)

test_true_mask = (~test_hivae_corrupted.isna()).astype(int)
_, _, hivae_decoded_array, _, _ = hivae_model.predict(
    test_hivae_corrupted, true_missing_mask=test_true_mask
)
hivae_decoded = pd.DataFrame(
    hivae_decoded_array, columns=test_hivae_corrupted.columns, index=test_hivae_corrupted.index
)
hivae_decoded_original = hivae_decoded.copy()
for column, categories in cat_maps.items():
    if column in hivae_decoded_original.columns:
        mapping = dict(enumerate(categories))
        hivae_decoded_original[column] = (
            hivae_decoded_original[column]
            .round()
            .clip(lower=0, upper=len(mapping) - 1)
            .astype(int)
            .map(mapping)
            .astype(str)
        )
for column, categories in ord_maps.items():
    if column in hivae_decoded_original.columns:
        mapping = dict(enumerate(categories))
        hivae_decoded_original[column] = (
            hivae_decoded_original[column]
            .round()
            .clip(lower=0, upper=len(mapping) - 1)
            .astype(int)
            .map(mapping)
            .astype(str)
        )

discrete_cols = sorted({*cat_maps.keys(), *ord_maps.keys()})
metrics_suave = evaluate_imputations(
    test_reference, suave_decoded, artificial_mask, discrete_cols
)
metrics_suave['model'] = 'SUAVE (behaviour="hivae")'
metrics_hivae = evaluate_imputations(
    test_reference, hivae_decoded_original, artificial_mask, discrete_cols
)
metrics_hivae['model'] = 'TensorFlow HI-VAE'
comparison = pd.concat([metrics_suave, metrics_hivae], ignore_index=True)
comparison


DEBUG:   	 1 2 hivae:_training:len(training_data) 925
DEBUG:   	 1 2 [*] Importing model: model_HIVAE_inputDropout
DEBUG:   	 1 2 [*] Defining placeholders


DEBUG:   	 1 2 [*] Defining Encoder...
DEBUG:   	 1 2 [*] Defining Decoder...


/workspace/SUAVE/third_party/hivae_tf/hivae/VAE_functions.py:117: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  log_pi = tf.compat.v1.layers.dense(inputs=X, units=s_dim, activation=None,
/workspace/SUAVE/third_party/hivae_tf/hivae/VAE_functions.py:134: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  mean_qz = tf.compat.v1.layers.dense(inputs=tf.concat([X,samples_s],1), units=z_dim, activation=None,
/workspace/SUAVE/third_party/hivae_tf/hivae/VAE_functions.py:136: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  log_var_qz = tf.compat.v1.layers.dense(inputs=tf.concat([X,samples_s],1), units=z_dim, activation=None,
/workspace/SUAVE/third_party/hivae_tf/hivae/VAE_functions.py:211: UserWarning: `tf.layers.dense` is deprecated and will be rem

/workspace/SUAVE/third_party/hivae_tf/hivae/VAE_functions.py:433: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  obs_output = tf.compat.v1.layers.dense(inputs=observed_data, units=output_dim, activation=None,
/workspace/SUAVE/third_party/hivae_tf/hivae/VAE_functions.py:435: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  miss_output = tf.compat.v1.layers.dense(inputs=missing_data, units=output_dim, activation=None,


DEBUG:   	 1 2 [*] Defining Cost function...


...

INFO:    	 0 3 Testing the HIVAE ...
INFO:    	 0 3 Restoring Model ...


INFO:    	 0 3 Model restored (examples/.cache/mimic_hivae_tf/networks/model_HIVAE_inputDropout_s32_z32_y32_batch64_0216dbd3-0e2c-419c-ac55-205bb0f3fa23_ckpt)


DEBUG:   	 1 2 Clusters: 10
DEBUG:   	 1 2 Epoch:    0	time: 4.44	train_loglik: -90.00	KL_z:  2.83	KL_s:  2.60	ELBO: -95.43	Test_loglik: 100000.00
INFO:    	 0 3 Testing Finished ...
DEBUG:   	 1 2 Saving reconstructions ...
DEBUG:   	 1 2 Saving model_HIVAE_inputDropout_s32_z32_y32_batch64_0216dbd3-0e2c-419c-ac55-205bb0f3fa23_data_reconstruction.csv in examples/.cache/mimic_hivae_tf/results/model_HIVAE_inputDropout_s32_z32_y32_batch64_0216dbd3-0e2c-419c-ac55-205bb0f3fa23
DEBUG:   	 1 2 Saving model_HIVAE_inputDropout_s32_z32_y32_batch64_0216dbd3-0e2c-419c-ac55-205bb0f3fa23_data_true.csv in examples/.cache/mimic_hivae_tf/results/model_HIVAE_inputDropout_s32_z32_y32_batch64_0216dbd3-0e2c-419c-ac55-205bb0f3fa23
DEBUG:   	 1 2 Saving model_HIVAE_inputDropout_s32_z32_y32_batch64_0216dbd3-0e2c-419c-ac55-205bb0f3fa23_data_loglik_mean_reconstructed.csv in examples/.cache/mimic_hivae_tf/results/model_HIVAE_inputDropout_s32_z32_y32_batch64_0216dbd3-0e2c-419c-ac55-205bb0f3fa23


DEBUG:   	 1 2 Reconstruction Correlation:
DEBUG:   	 1 2 0     0.686091
1     0.457721
2     0.590716
3     0.148674
4     0.024462
5     0.288310
6     0.396587
7     0.328704
8     0.437705
9     0.365109
10    0.013316
11         NaN
12    0.727907
13    0.529869
14    0.381945
15    0.162274
16    0.389432
17    0.553487
18    0.523735
19    0.534773
20    0.508565
21    0.655700
22    0.620071
23    0.440944
24    0.264637
25    0.062423
26   -0.008305
27    0.419456
28    0.418638
29   -0.140631
30    0.579155
31    0.529854
32    0.387302
33    0.420514
34    0.427708
35    0.820980
36    0.813926
dtype: float64


,column,metric,value,model
0,Unnamed: 0,mse,1.346474e+08,"SUAVE (behaviour=""hivae"")"
1,age,mse,3.939931e+02,"SUAVE (behaviour=""hivae"")"
2,sex,categorical_accuracy,5.326087e-01,"SUAVE (behaviour=""hivae"")"
3,BMI,mse,2.052549e+02,"SUAVE (behaviour=""hivae"")"
4,temperature,mse,4.661275e-01,"SUAVE (behaviour=""hivae"")"
...,...,...,...,...
69,PaCO2,mse,1.239846e+02,TensorFlow HI-VAE
70,HCO3-,mse,1.495768e+01,TensorFlow HI-VAE
71,Lac,mse,8.874465e-01,TensorFlow HI-VAE
72,3d_septic_shock,categorical_accuracy,1.000000e+00,TensorFlow HI-VAE


In [13]:

summary_metrics = comparison.groupby(['model', 'metric'])['value'].mean().reset_index()
summary_metrics


,model,metric,value
0,"SUAVE (behaviour=""hivae"")",categorical_accuracy,6.394928e-01
1,"SUAVE (behaviour=""hivae"")",mse,4.352337e+06
2,TensorFlow HI-VAE,categorical_accuracy,7.699275e-01
3,TensorFlow HI-VAE,mse,2.925877e+06


In [14]:

wide_view = comparison.pivot_table(
    index=['column', 'metric'], columns='model', values='value'
)
wide_view


,model,"SUAVE (behaviour=""hivae"")",TensorFlow HI-VAE
column,metric,,
3d_septic_shock,categorical_accuracy,8.695652e-01,1.000000e+00
7d_septic_shock,categorical_accuracy,8.260870e-01,9.673913e-01
ALT,mse,6.462707e+04,2.305540e+04
APTT,mse,3.712128e+02,1.465725e+02
AST,mse,1.147919e+05,4.681607e+04
BMI,mse,2.052549e+02,9.180500e+01
BUN,mse,1.263479e+02,5.838893e+01
CRRT,categorical_accuracy,9.130435e-01,1.000000e+00
DBP,mse,1.963379e+02,5.422391e+01


In [15]:

differences = []
for column in test_reference.columns:
    if column in discrete_cols:
        suave_values = suave_decoded[column].astype(str)
        hivae_values = hivae_decoded_original[column].astype(str)
        agreement = float((suave_values == hivae_values).mean())
        differences.append({'column': column, 'metric': 'agreement', 'value': agreement})
    else:
        suave_numeric = pd.to_numeric(suave_decoded[column], errors='coerce')
        hivae_numeric = pd.to_numeric(hivae_decoded_original[column], errors='coerce')
        diff = suave_numeric - hivae_numeric
        mae = float(np.nanmean(np.abs(diff)))
        differences.append({'column': column, 'metric': 'mean_abs_diff', 'value': mae})
pd.DataFrame(differences)


,column,metric,value
0,Unnamed: 0,mean_abs_diff,7204.856054
1,age,mean_abs_diff,12.770270
2,sex,agreement,0.515676
3,BMI,mean_abs_diff,5.699933
4,temperature,mean_abs_diff,0.462132
5,heart_rate,mean_abs_diff,12.660697
6,respir_rate,mean_abs_diff,3.492576
7,SBP,mean_abs_diff,12.559832
8,DBP,mean_abs_diff,9.480161
9,MAP,mean_abs_diff,9.244594


In [16]:

preview_columns = list(discrete_cols)[:3] + [col for col in test_reference.columns if col not in discrete_cols][:3]
preview = pd.concat(
    {
        'original': test_reference[preview_columns],
        'suave_imputed': suave_decoded[preview_columns],
        'hivae_imputed': hivae_decoded_original[preview_columns],
    },
    axis=1
)
preview.head()


original                                                 \
  3d_septic_shock 7d_septic_shock CRRT Unnamed: 0 age        BMI   
0               0               0    0       9686  76  35.967860   
1               0               0    0      26289  78  23.979239   
2               0               0    0      14629  91        NaN   
3               0               0    0      11602  91  23.154800   
4               0               0    0      25105  74        NaN   

    suave_imputed                                                           \
  3d_septic_shock 7d_septic_shock CRRT    Unnamed: 0        age        BMI   
0               0               0    0  34315.453125  76.366600  13.719404   
1               0               0    0  17530.562500  65.365295  36.954567   
2               0               1    0  15548.692383  71.511894  25.371410   
3               0               1    0  19247.982422  83.804085  32.788475   
4               0               0    0  28334.058594  88.260078  33.323540   

    hivae_imputed                                                           
  3d_septic_shock 7d_septic_shock CRRT    Unnamed: 0        age        BMI  
0               0               0    0  12603.884766  69.379837  31.367161  
1               0               0    0  16884.281250  72.872253  25.499931  
2               0               0    0  15299.752930  64.916603  28.279955  
3               0               0    0  13216.267578  70.672272  25.346279  
4               0               0    0  17318.648438  74.553017  27.915478

In [17]:

try:
    suave_model.predict(test_suave_corrupted)
except RuntimeError as exc:
    print('SUAVE predict() error:', exc)

print('TensorFlow HI-VAE predict() callable:', callable(getattr(hivae_model, 'predict', None)))


SUAVE predict() error: predict is unavailable when behaviour='hivae'; this mode matches the baseline HI-VAE and does not expose classifier outputs.
TensorFlow HI-VAE predict() callable: True



## 小结

* 表格对比显示两种实现对于数值变量的均方误差 (MSE) 与类别变量的准确率非常接近。
* 直接比较 SUAVE 与 TensorFlow HI-VAE 的重建输出，数值变量的平均绝对差仅为很小的量级，类别变量几乎完全一致。
* 当 `behaviour='hivae'` 时，SUAVE 遵循原始 HI-VAE 设计，不提供分类 `predict()` 接口，因此与 TensorFlow 参考实现的 `predict` (用于生成重建结果) 在功能上保持一致。
